In [1]:
import pandas as pd
import numpy as np

from catboost import CatBoostClassifier

from sklearn import metrics


from sklearn.metrics import make_scorer

from sklearn.metrics import roc_auc_score




In [2]:
train = pd.read_csv("C:/Users/DELL/Desktop/train.csv")

In [3]:
train.head(10)

,ID,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium,Response
0,1,C3,3213,Rented,Individual,36,36,No,X1,14+,3.0,22,11628.0,0
1,2,C5,1117,Owned,Joint,75,22,No,X2,NaN,NaN,22,30510.0,0
2,3,C5,3732,Owned,Individual,32,32,No,NaN,1.0,1.0,19,7450.0,1
3,4,C24,4378,Owned,Joint,52,48,No,X1,14+,3.0,19,17780.0,0
4,5,C8,2190,Rented,Individual,44,44,No,X2,3.0,1.0,16,10404.0,0
5,6,C9,1785,Rented,Individual,52,52,No,X2,5.0,1.0,22,15264.0,1
6,7,C3,679,Owned,Individual,28,28,No,NaN,NaN,NaN,17,10640.0,0
7,8,C1,3175,Owned,Joint,75,73,Yes,X4,9.0,4.0,17,29344.0,1
8,9,C15,3497,Owned,Joint,52,43,No,X1,14.0,3.0,1,27283.2,0
9,10,C1,530,Owned,Joint,59,26,Yes,NaN,7.0,4.0,18,21100.8,1


In [4]:
train.fillna({'Health Indicator':'None', 'Holding_Policy_Duration':'None','Holding_Policy_Type':'None'}, inplace=True)

In [5]:
train.isnull().sum()

ID                         0
City_Code                  0
Region_Code                0
Accomodation_Type          0
Reco_Insurance_Type        0
Upper_Age                  0
Lower_Age                  0
Is_Spouse                  0
Health Indicator           0
Holding_Policy_Duration    0
Holding_Policy_Type        0
Reco_Policy_Cat            0
Reco_Policy_Premium        0
Response                   0
dtype: int64

In [6]:
test = pd.read_csv('C:/Users/DELL/Desktop/test.csv')
test.fillna({'Health Indicator':'None','Holding_Policy_Duration':'None','Holding_Policy_Type':'None'}, inplace=True)
test = test.drop(['ID'],axis=1)

In [7]:
train = train.drop(['ID'],axis=1)
test['Response'] = -1

data_1 = pd.concat([train,test],axis=0)

data_1['Reco_Policy_Cat']= data_1['Reco_Policy_Cat'].astype('int')



In [8]:
data_1['difference'] = data_1['Upper_Age'] - data_1['Lower_Age']
data_1['Ratio'] = data_1['Reco_Policy_Premium'] / data_1['Upper_Age']


In [9]:
mappings = { 'X1' : 0, 'X2' : 0,'X3' : 0, 'X4' : 1, 'X5' : 1, 'X6' : 1, 'X7' : 2, 'X8' : 2, 'X9' : 2}

mapping = {'14+' : 2, '1.0' : 0, '3.0': 0, '5.0': 0, '9.0': 1, '14.0': 2, '7.0': 1, '2.0': 0,
       '11.0': 2, '10.0': 1, '8.0': 1, '6.0' : 1, '4.0': 0, '13.0': 2, '12.0': 2}

label_ranges = [0, 25, 50,80]

label_names = [0,1,2]

label_range = [x for x in range(0,46000,2000)]

label_name = [x for x in range(22)]


data_1['Upper_Age'] = pd.cut(data_1['Upper_Age'], bins = label_ranges, labels = label_names)

data_1['Lower_Age'] = pd.cut(data_1['Lower_Age'], bins = label_ranges, labels = label_names)

data_1['Health Indicator'] = data_1['Health Indicator'].replace(mappings)
data_1['Holding_Policy_Duration'] = data_1['Holding_Policy_Duration'].replace(mapping)


data_1['Reco_Policy_Premium'] = pd.cut(data_1['Reco_Policy_Premium'], bins = label_range, labels = label_name)


In [10]:
c = ['City_Code', 'Region_Code', 'Accomodation_Type',
       'Reco_Insurance_Type', 'Is_Spouse',
       'Health Indicator', 'Holding_Policy_Duration', 'Holding_Policy_Type',
       'Reco_Policy_Cat']

for i in c:
    data_1[i] = data_1[i].astype('str')
    

In [11]:
e = data_1.columns

e = ['City_Code', 'Region_Code', 'Accomodation_Type',
       'Reco_Insurance_Type', 'Is_Spouse',
       'Health Indicator', 'Holding_Policy_Duration', 'Holding_Policy_Type',
       'Reco_Policy_Cat',
       'City_Code_Region_Code', 'City_Code_Accomodation_Type',
       'City_Code_Reco_Insurance_Type', 'City_Code_Is_Spouse',
       'City_Code_Health Indicator', 'City_Code_Holding_Policy_Duration',
       'City_Code_Holding_Policy_Type', 'City_Code_Reco_Policy_Cat',
       'Region_Code_Accomodation_Type', 'Region_Code_Reco_Insurance_Type',
       'Region_Code_Is_Spouse', 'Region_Code_Health Indicator',
       'Region_Code_Holding_Policy_Duration',
       'Region_Code_Holding_Policy_Type', 'Region_Code_Reco_Policy_Cat',
       'Accomodation_Type_Reco_Insurance_Type', 'Accomodation_Type_Is_Spouse',
       'Accomodation_Type_Health Indicator',
       'Accomodation_Type_Holding_Policy_Duration',
       'Accomodation_Type_Holding_Policy_Type',
       'Accomodation_Type_Reco_Policy_Cat', 'Reco_Insurance_Type_Is_Spouse',
       'Reco_Insurance_Type_Health Indicator',
       'Reco_Insurance_Type_Holding_Policy_Duration',
       'Reco_Insurance_Type_Holding_Policy_Type',
       'Reco_Insurance_Type_Reco_Policy_Cat', 'Is_Spouse_Health Indicator',
       'Is_Spouse_Holding_Policy_Duration', 'Is_Spouse_Holding_Policy_Type',
       'Is_Spouse_Reco_Policy_Cat', 'Health Indicator_Holding_Policy_Duration',
       'Health Indicator_Holding_Policy_Type',
       'Health Indicator_Reco_Policy_Cat',
       'Holding_Policy_Duration_Holding_Policy_Type',
       'Holding_Policy_Duration_Reco_Policy_Cat',
       'Holding_Policy_Type_Reco_Policy_Cat']

In [12]:
from itertools import combinations 

d = combinations(c,2)

for i, j in d:
    data_1[i + '_' + j] = data_1[i].astype(str) + '_' + data_1[j].astype(str)
    
data_1.head()

,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,...,Is_Spouse_Health Indicator,Is_Spouse_Holding_Policy_Duration,Is_Spouse_Holding_Policy_Type,Is_Spouse_Reco_Policy_Cat,Health Indicator_Holding_Policy_Duration,Health Indicator_Holding_Policy_Type,Health Indicator_Reco_Policy_Cat,Holding_Policy_Duration_Holding_Policy_Type,Holding_Policy_Duration_Reco_Policy_Cat,Holding_Policy_Type_Reco_Policy_Cat
0,C3,3213,Rented,Individual,1,1,No,0,2,3.0,...,No_0,No_2,No_3.0,No_22,0_2,0_3.0,0_22,2_3.0,2_22,3.0_22
1,C5,1117,Owned,Joint,2,0,No,0,None,None,...,No_0,No_None,No_None,No_22,0_None,0_None,0_22,None_None,None_22,None_22
2,C5,3732,Owned,Individual,1,1,No,None,0,1.0,...,No_None,No_0,No_1.0,No_19,None_0,None_1.0,None_19,0_1.0,0_19,1.0_19
3,C24,4378,Owned,Joint,2,1,No,0,2,3.0,...,No_0,No_2,No_3.0,No_19,0_2,0_3.0,0_19,2_3.0,2_19,3.0_19
4,C8,2190,Rented,Individual,1,1,No,0,0,1.0,...,No_0,No_0,No_1.0,No_16,0_0,0_1.0,0_16,0_1.0,0_16,1.0_16


In [13]:
from sklearn.preprocessing import OrdinalEncoder
ord_enc = OrdinalEncoder()

for i in e:
    data_1[i] = ord_enc.fit_transform(data_1[[i]])

data_1.shape

(72687, 51)

In [14]:
X = data_1.drop(['Response'],axis=1)
y = data_1['Response']


In [15]:
from sklearn import model_selection

x_e, x_p, y_e, y_t = model_selection.train_test_split(X,y, test_size = 0.25, shuffle = True,random_state = 22)



In [16]:
imp = [
 'Region_Code_Reco_Policy_Cat',
 'Region_Code_Holding_Policy_Duration',
 'Region_Code_Health Indicator',
 'Region_Code_Holding_Policy_Type',
 'Region_Code_Accomodation_Type',
 'Region_Code_Reco_Insurance_Type',
 'Region_Code_Is_Spouse',
 'Reco_Policy_Premium',
 'Region_Code',
 'City_Code_Region_Code',
 'City_Code_Reco_Policy_Cat',
 'Holding_Policy_Type_Reco_Policy_Cat',
 'Holding_Policy_Duration_Reco_Policy_Cat',
 'Health Indicator_Reco_Policy_Cat',
 'Accomodation_Type_Reco_Policy_Cat',
 'Is_Spouse_Reco_Policy_Cat',
 'Reco_Insurance_Type_Reco_Policy_Cat',
 'Reco_Policy_Cat']

In [17]:
from category_encoders import MEstimateEncoder

# Create the encoder instance. Choose m to control noise.
encoder = MEstimateEncoder(cols = imp, m=5)

# Fit the encoder on the encoding split.
encoder.fit(x_e, y_e)

# Encode the Zipcode column to create the final training data
X_train = encoder.transform(x_p)


C:\Users\DELL\Anaconda3\envs\PythonML\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [18]:
# x = X_train.drop(['Response'],axis=1)
# y = data_1['Response']
x = pd.concat([x_e,X_train],axis=0)

y = pd.concat([y_e,y_t],axis=0)

e = x.dtypes != int

from sklearn.feature_selection import mutual_info_classif 

def make_mi_scores(x, y, e):
    mi_scores = mutual_info_classif(x, y, discrete_features=e)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=x.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

mi_scores = make_mi_scores(x, y, e)
mi_scores[::3]  # show a few features with their MI scores




C:\Users\DELL\Anaconda3\envs\PythonML\lib\site-packages\sklearn\metrics\cluster\_supervised.py:58: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and multiclass values for target
  warnings.warn(msg, UserWarning)
C:\Users\DELL\Anaconda3\envs\PythonML\lib\site-packages\sklearn\metrics\cluster\_supervised.py:58: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and multiclass values for target
  warnings.warn(msg, UserWarning)
C:\Users\DELL\Anaconda3\envs\PythonML\lib\site-packages\sklearn\metrics\cluster\_supervised.py:58: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and multiclass values for target
  warnings.warn(msg, UserWarning)
C:\Users\DELL\Anaconda3\envs\PythonML\lib\site-packages\sklearn\metrics\cluster\_supervised.py:58: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and multicl

Ratio                                          0.475208
Region_Code_Holding_Policy_Duration            0.244496
Region_Code_Reco_Insurance_Type                0.143457
Region_Code                                    0.088523
Holding_Policy_Duration_Reco_Policy_Cat        0.018707
Is_Spouse_Reco_Policy_Cat                      0.017658
City_Code_Holding_Policy_Type                  0.002340
City_Code_Accomodation_Type                    0.001055
difference                                     0.000840
Health Indicator_Holding_Policy_Type           0.000402
Reco_Insurance_Type_Holding_Policy_Duration    0.000200
Accomodation_Type_Health Indicator             0.000158
Reco_Insurance_Type_Health Indicator           0.000109
Accomodation_Type_Holding_Policy_Type          0.000103
Is_Spouse_Health Indicator                     0.000080
Accomodation_Type_Is_Spouse                    0.000067
Accomodation_Type                              0.000021
Name: MI Scores, dtype: float64

In [19]:
imp_columns = mi_scores.index[:25]
imp_columns = list(imp_columns)
imp_columns.extend(['Response','Reco_Policy_Premium'])
imp_columns

data_1 = data_1[imp_columns]

data_1.head()




,Ratio,Region_Code_Reco_Policy_Cat,Region_Code_Holding_Policy_Type,Region_Code_Holding_Policy_Duration,Region_Code_Health Indicator,Region_Code_Accomodation_Type,Region_Code_Reco_Insurance_Type,Region_Code_Is_Spouse,City_Code_Region_Code,Region_Code,...,Reco_Policy_Cat,City_Code_Holding_Policy_Type,City_Code_Holding_Policy_Duration,City_Code_Health Indicator,City_Code_Accomodation_Type,City_Code_Reco_Insurance_Type,City_Code_Is_Spouse,difference,Response,Reco_Policy_Premium
0,323.000000,13178.0,9179.0,7953.0,6489.0,4259.0,4156.0,4065.0,3950.0,2137.0,...,14.0,146.0,118.0,113.0,59.0,58.0,58.0,0,0,5
1,406.800000,828.0,530.0,445.0,362.0,230.0,228.0,226.0,4521.0,117.0,...,14.0,158.0,127.0,121.0,62.0,63.0,62.0,53,0,15
2,232.812500,15692.0,11194.0,9747.0,7936.0,5298.0,5118.0,4994.0,4588.0,2664.0,...,10.0,154.0,124.0,124.0,62.0,62.0,62.0,0,1,3
3,341.923077,18426.0,13445.0,11838.0,9687.0,6573.0,6279.0,6115.0,2448.0,3322.0,...,10.0,77.0,62.0,60.0,30.0,31.0,30.0,4,0,8
4,236.454545,7640.0,5093.0,4381.0,3563.0,2291.0,2264.0,2226.0,5124.0,1151.0,...,7.0,169.0,136.0,133.0,69.0,68.0,68.0,0,0,5


In [20]:
test = data_1[data_1['Response']== -1]

train = data_1[data_1['Response']!= -1]


y = train['Response'].values
x = train.drop(['Response'],axis=1).values

test.drop('Response',axis=1,inplace=True)



C:\Users\DELL\Anaconda3\envs\PythonML\lib\site-packages\pandas\core\frame.py:4164: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [21]:
from sklearn import model_selection

x_train, x_test, y_train, y_test = model_selection.train_test_split(x,y,stratify = y, test_size = 0.20, shuffle = True,random_state = 22)



In [22]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import StackingClassifier

rf = RandomForestClassifier(max_depth=16,max_features = 'auto',n_jobs = -1,random_state=22)

cbc = CatBoostClassifier(verbose = 0)

xgb = XGBClassifier(n_jobs = -1,verbose = 0)


In [23]:
from sklearn.linear_model import LogisticRegression

est = [('xgb',xgb),('rf',rf),('cbc',cbc)]
stacker = StackingClassifier(estimators = est,final_estimator = LogisticRegression(),cv = 5)
stacker.fit(x_train,y_train)

pred = predictions[:,1]

sample = pd.read_csv('C:/Users/DELL/Desktop/sample.csv')

sample['Response'] = pred

sample.to_csv('submission_stack.csv',index=False)

# st1 = stacker.predict_proba(x_train)
# st2 = stacker.predict_proba(x_test)
# print('train_score = ',roc_auc_score(y_train,st1[:,1]))
# print('test_score = ',roc_auc_score(y_test,st2[:,1]))

C:\Users\DELL\Anaconda3\envs\PythonML\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:26:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { verbose } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[22:26:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\DELL\Anaconda3\envs\PythonML\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:27:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { verbose } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[22:27:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\DELL\Anaconda3\envs\PythonML\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

C:\Users\DELL\Anaconda3\envs\PythonML\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:27:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { verbose } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[22:27:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\DELL\Anaconda3\envs\PythonML\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:27:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { verbose } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[22:27:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


NameError: name 'predictions' is not defined

In [ ]:

# from sklearn.ensemble import RandomForestClassifier

# # Instantiate a DecisionTreeClassifier 'dt' with a maximum depth of 6
# rf = RandomForestClassifier(max_depth=16,max_features = 'auto',n_jobs = -1,random_state=22)

# # Fit dt to the training set
# rf.fit(x, y)

# # Predict test set labels
# #y_pred = dt.predict_proba(x)




# from sklearn.metrics import roc_auc_score

# predictions = rf.predict_proba(test)


# #print (roc_auc_score(y, predictions[:,1]))

# pred = predictions[:,1]

# sample = pd.read_csv('C:/Users/DELL/Desktop/sample.csv')

# sample['Response'] = pred

# sample.to_csv('submission_final.csv',index=False)



In [ ]:
# modelC = CatBoostClassifier(verbose = 0)
# modelC = modelC.fit(x_train, y_train)

# predictions = [pred[1] for pred in modelC.predict_proba(x_test)]
# print('Validation ROC AUC Score:', roc_auc_score(y_test, predictions, average = 'weighted'))

In [ ]:
# Import DecisionTreeClassifier from sklearn.tree
# from sklearn.ensemble import RandomForestClassifier

# # Instantiate a DecisionTreeClassifier 'dt' with a maximum depth of 6
# rf = RandomForestClassifier(max_depth = 16,max_features = 'auto',n_jobs = -1,random_state=22)

# # Fit dt to the training set
# rf.fit(x_train, y_train)

# # Predict test set labels
# #y_pred = dt.predict_proba(x)




# from sklearn.metrics import roc_auc_score

# predictions = rf.predict_proba(x_test)

# print (roc_auc_score(y_test, predictions[:,1]))



In [ ]:
# from xgboost import XGBClassifier

# xgb = XGBClassifier(n_jobs = -1,verbose = 0)

# xgb.fit(x_train, y_train)

# from sklearn.metrics import roc_auc_score

# predictions = xgb.predict_proba(x_test)

# print (roc_auc_score(y_test, predictions[:,1]))

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import roc_auc_score
# n_estimators = [100, 300, 500, 800, 1200]
# max_depth = [5, 8, 15, 25, 30]
# min_samples_split = [2, 5, 10, 15, 100]
# min_samples_leaf = [1, 2, 5, 10] 

# # hyperF = dict(n_estimators = n_estimators, max_depth = max_depth,  
#               min_samples_split = min_samples_split, 
#              min_samples_leaf = min_samples_leaf)

# rf = RandomForestClassifier()

# gridF = GridSearchCV(rf, hyperF, cv = 3, verbose = 1, 
#                       n_jobs = -1,scoring = 'roc_auc')
# gridF.fit(x_train, y_train)

In [ ]:
# gridf.best_params_

# best_model = gridf.best_estimator_

# predictions = best_model.predict_proba(x_test)

# print (roc_auc_score(y_test, predictions[:,1]))



In [ ]:
# modelC = CatBoostClassifier()
# modelC = modelC.fit(x, y)

# predictions = modelC.predict_proba(test)

# pred = predictions[:,1]

# sample = pd.read_csv('sample.csv')

# sample['Response'] = pred

# sample.to_csv('submission8.csv',index=False)